# FOM - Anwendungsfelder der Business Analytics am Beispiel von Fraud Detection im von Kreditkartentransaktionen
In diesem Jupyter Note soll ein künstliches neuronales Netz (KNN) erstellt werden, mit welchem Kreditkartentransaktionen auf betrügerische Aktivitäten hin überprüft werden. Für die Umsetzung dieses Vorhabens sollen die in dem Notebook <code>ieee-cis-k-means</code> erstellten Daten und Tensorflow verwendet werden.</br></br>
<b>Folgende Schritte sind zu erledigen</b>
<ol>
    <li>Import der nötigen Python Packages</li>
    <li>Import der vorbereiteten Daten aus AWS S3</li>
    <li>Normalisierung der importierten Daten</li>
</ol>
</br>
Im folgenden werden die zu verwenden Python Packages importiert.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import boto3
import io
import matplotlib.pyplot as plt
import numpy as py
import pandas as pd
import tensorflow as tf

In [2]:
print('{}: {}'.format('Tensorflow version', tf.__version__))

Tensorflow version: 1.14.0


Nachdem die nötigen Python Packages importiert sind, können die zuletzt vorbereiteten Daten aus AWS S3 importiert werden.
<br>
## Import der vorbereiteten Daten aus AWS S3
Für den Import von Daten aus AWS S3 muss zunächst der AWS S3 client initialisiert und der entsprechende Bucket Name angegeben werden.

In [3]:
s3_client = boto3.client('s3')
bucket_name = 'anwendungsfelder-big-data-fraud-detection-ieee-cis-data-set'

Anschließend wird eine Iteration über alle in dem AWS S3 Bucket enthaltende Objekte vorgenommen.

In [4]:
obj_list = s3_client.list_objects(Bucket=bucket_name)
for obj in obj_list:
    print(obj)

Name
ResponseMetadata
MaxKeys
Prefix
Marker
EncodingType
IsTruncated
Contents


Da der Inhalt des Buckets benötigt wird, wird eine Iteration über den Inhalt des Buckets durchgeführt. Da aber das aus der Abfrage zurückkommende Objekt nicht bekannt ist, wird zunächst nur ein Element aus der Liste ausgegeben.

In [5]:
content_list = obj_list['Contents']
print(content_list[0])

{u'LastModified': datetime.datetime(2019, 11, 8, 14, 34, 4, tzinfo=tzlocal()), u'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', u'StorageClass': 'STANDARD', u'Key': u'data/', u'Size': 0}


<code>Key</code> scheint den relativen Pfad zu einem Objekt darzustellen. Im folgenden wird eine Iteration über diesen Key vorgenommen.

In [6]:
i = 0
file_to_retrieve = 'clustering_result.csv'
for content_element in content_list:
    if file_to_retrieve in content_element['Key']: 
        print('{}: {}'.format(str(i), content_element['Key']))
    i += 1

1: data/clustering_result.csv


In [7]:
file_list = []
for content_element in content_list:
    if file_to_retrieve in content_element['Key']:
        file_list.append(content_element['Key'])
print('In Liste vorhandene Element: {}'.format(str(len(file_list))))

In Liste vorhandene Element: 1


In [8]:
s3_file = s3_client.get_object(Bucket=bucket_name, Key=file_list[0])
print('Type of object: {}'.format(str(type(s3_file))))

Type of object: <type 'dict'>


In [9]:
print(s3_file)

{u'Body': <botocore.response.StreamingBody object at 0x7fb206a4dad0>, u'AcceptRanges': 'bytes', u'ContentType': 'binary/octet-stream', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'id1mL7NW9PMV4wuLtbXPyG6tned8CSHPa2FXhSpRwb2j7Nv9lwjORb5Ia96nXT+jnFYcWzDuIfU=', 'RequestId': '563F07BC8D4E2A87', 'HTTPHeaders': {'content-length': '19322227', 'x-amz-id-2': 'id1mL7NW9PMV4wuLtbXPyG6tned8CSHPa2FXhSpRwb2j7Nv9lwjORb5Ia96nXT+jnFYcWzDuIfU=', 'accept-ranges': 'bytes', 'server': 'AmazonS3', 'last-modified': 'Sun, 26 Jan 2020 17:08:04 GMT', 'x-amz-request-id': '563F07BC8D4E2A87', 'etag': '"79ad4dde2e2c759c52243eecf613ed88-3"', 'date': 'Fri, 31 Jan 2020 16:31:14 GMT', 'content-type': 'binary/octet-stream'}}, u'LastModified': datetime.datetime(2020, 1, 26, 17, 8, 4, tzinfo=tzutc()), u'ContentLength': 19322227, u'ETag': '"79ad4dde2e2c759c52243eecf613ed88-3"', u'Metadata': {}}


In [10]:
result_file = s3_file['Body'].read()
print('Type of object: {}'.format(str(type(result_file))))

Type of object: <type 'str'>


## Normalisierung der importierten Daten
Da die Daten vorhanden sind können diese in einem <code>DataFrame</code> dargestellt werden.

In [11]:
df = pd.read_csv(io.BytesIO(result_file), header=0)
df.head()

,credit_card,transaction_id,amount_spent,belonging_to_cluster
0,1000.0,3230924.0,23.443001,0.0
1,1001.0,3023634.0,183.000000,0.0
2,1001.0,3210739.0,27.000000,0.0
3,1001.0,3151336.0,29.000000,0.0
4,1004.0,3504379.0,226.000000,0.0


In [12]:
df.shape

(590549, 4)

Bevor die Daten normalisiert werden können, werden diese mit dem tag <code>if_Fraud</code> angereichert. Daher müssen noch weitere Daten importiert werden.

In [13]:
i = 0
file_to_retrieve_2 = 'final_cropped_data.csv'
for content_element in content_list:
    if file_to_retrieve_2 in content_element['Key']: 
        print('{}: {}'.format(str(i), content_element['Key']))
    i += 1

4: data/ieee_cis_train_cropped/Definite+cropped+ieeecis+data/2020/01/20/final_cropped_data.csv


In [14]:
file_list_2 = []
for content_element in content_list:
    if file_to_retrieve_2 in content_element['Key']:
        file_list_2.append(content_element['Key'])
print('In Liste vorhandene Element: {}'.format(str(len(file_list))))

In Liste vorhandene Element: 1


In [15]:
s3_file_2 = s3_client.get_object(Bucket=bucket_name, Key=file_list_2[0])
print('Type of object: {}'.format(str(type(s3_file))))

Type of object: <type 'dict'>


In [16]:
result_file_2 = s3_file_2['Body'].read()
print('Type of object: {}'.format(str(type(result_file))))

Type of object: <type 'str'>


In [17]:
df2 = pd.read_csv(io.BytesIO(result_file_2), header=0)
df2.head()

,credit_card_number,transaction_id,transaction_is_fraudulent,amount_spent,transaction_time_delta,credit_card_holder,credit_card_expiration_date,crdeit_card_validation_code_two
0,1000,3230924,0,23.443,5787419,555.0,185.0,224.0
1,1001,3023634,0,183.000,916268,555.0,150.0,226.0
2,1001,3210739,0,27.000,5270458,555.0,150.0,226.0
3,1001,3151336,0,29.000,3504180,555.0,150.0,226.0
4,1004,3504379,0,226.000,13553567,583.0,150.0,226.0


In [34]:
df2.shape

(590540, 6)

Da nun die Grundlage für die als erste importierten vorliegen, können nun die Datentypen angepasst werden.

In [18]:
df.dtypes

credit_card             float64
transaction_id          float64
amount_spent            float64
belonging_to_cluster    float64
dtype: object

In [19]:
df = df.astype({
    'credit_card' : 'int32',
    'transaction_id' : 'int32',
    })

Nun können die Daten nochmals angereichert werden.

In [20]:
df.head()

,credit_card,transaction_id,amount_spent,belonging_to_cluster
0,1000,3230924,23.443001,0.0
1,1001,3023634,183.000000,0.0
2,1001,3210739,27.000000,0.0
3,1001,3151336,29.000000,0.0
4,1004,3504379,226.000000,0.0


In [21]:
df2 = df2.drop(columns=[
    'credit_card_number',
    'amount_spent'
])

In [37]:
result_df = pd.merge(df, df2, on='transaction_id')
result_df = result_df.drop_duplicates()
result_df.head(10)

,credit_card,transaction_id,amount_spent,belonging_to_cluster,transaction_is_fraudulent,transaction_time_delta,credit_card_holder,credit_card_expiration_date,crdeit_card_validation_code_two
0,1000,3230924,23.443001,0.0,0,5787419,555.0,185.0,224.0
2,1001,3023634,183.000000,0.0,0,916268,555.0,150.0,226.0
4,1001,3210739,27.000000,0.0,0,5270458,555.0,150.0,226.0
6,1001,3151336,29.000000,0.0,0,3504180,555.0,150.0,226.0
8,1004,3504379,226.000000,0.0,0,13553567,583.0,150.0,226.0
10,1004,3386444,50.000000,0.0,0,10082484,583.0,150.0,226.0
12,1004,3028973,30.000000,0.0,0,1022173,583.0,150.0,226.0
14,1004,3504371,226.000000,0.0,0,13553346,583.0,150.0,226.0
16,1004,3020767,150.000000,0.0,0,842821,583.0,150.0,226.0
18,1005,3038871,50.000000,0.0,0,1212802,543.0,150.0,117.0


In [23]:
result_df.shape

(590540, 9)